In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
plt.style.use('fivethirtyeight') 
from numpy.random import normal, seed
from scipy.stats import norm
from datetime import datetime
from sklearn.preprocessing import LabelBinarizer,LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
rcParams['figure.figsize'] = 10, 6
import os
import warnings
warnings.filterwarnings('ignore')
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import math
print(os.listdir("./Data/"))

['Clean_test_data.csv', 'Clean_train_data.csv', 'holidays.xlsx', 'processed_test.csv', 'processed_train.csv', 'promotional_expense.csv', 'sample_submission.csv', 'yds_test2018.csv', 'yds_train2018.csv']


In [3]:
import xgboost

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train = pd.read_csv('Data/processed_train.csv')
test = pd.read_csv('Data/processed_test.csv')

In [6]:
train.head()

,Year,Sales,Holiday,Expense_Price,Country0,Country1,Country2,Country3,Country4,Country5,Product_ID0,Product_ID1,Product_ID2,Product_ID3,Product_ID4,sin_month,cos_month
0,0.0,7.535876,0.25,4.168801,1,0,0,0,0,0,1,0,0,0,0,0.5,0.866025
1,0.0,6.439625,0.25,3.123974,1,0,0,0,0,0,0,1,0,0,0,0.5,0.866025
2,0.0,5.498105,0.25,2.398964,0,1,0,0,0,0,0,1,0,0,0,0.5,0.866025
3,0.0,6.897336,0.50,3.277408,0,0,1,0,0,0,1,0,0,0,0,0.5,0.866025
4,0.0,6.761197,0.50,3.157676,0,0,1,0,0,0,0,1,0,0,0,0.5,0.866025


In [7]:
train.shape

(388, 17)

In [9]:
tr = train.iloc[:math.floor(388*0.8)]
val = train.iloc[math.ceil(388*0.8):]

In [17]:
xgr = xgboost.XGBRegressor(n_estimators=150,learning_rate=0.05,max_depth=30)

In [18]:
x_tr = tr.drop(columns=['Sales'])
y_tr = tr.Sales

In [21]:
x_val = val.drop(columns=['Sales'])
y_val = val.Sales

In [19]:
model = xgr.fit(x_tr,y_tr)

[17:23:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=30, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [22]:
y_tr_pred = model.predict(x_tr)
y_val_pred = model.predict(x_val)

In [26]:
compare_tr = pd.DataFrame({'Orig':y_tr,'Pred':y_tr_pred})

In [25]:
compare_tr.Orig.corr(compare_tr.Pred)

0.9999378599736235

# SMAPE

In [29]:
numerator = (compare_tr.Orig - compare_tr.Pred).abs()
denominator = (compare_tr.Orig + compare_tr.Pred).abs()
count = numerator.count()
out = (numerator/denominator).sum()
SMAPE_tr = (out/count)*100
print('SMAPE of training : ',SMAPE_tr)

SMAPE of training :  0.05311518348111784


In [30]:
compare_val = pd.DataFrame({'Orig':y_val,'Pred':y_val_pred})

In [31]:
compare_val.Orig.corr(compare_val.Pred)

0.7563990832956984

In [39]:
def SMAPE(compare_val):
    numerator = (compare_val.Orig - compare_val.Pred).abs()
    denominator = (compare_val.Orig + compare_val.Pred).abs()
    count = numerator.count()
    out = (numerator/denominator).sum()
    SMAPE = (out/count)*100
    print('SMAPE : ',SMAPE)

In [33]:
xTrain = train.drop(columns=['Sales'])
yTrain = train.Sales

In [34]:
model = xgr.fit(xTrain,yTrain)

[17:35:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [35]:
y_Predict = xgr.predict(xTrain)

In [36]:
compareTrain = pd.DataFrame({'Orig':yTrain,'Pred':y_Predict})

In [38]:
compareTrain.Orig.corr(compareTrain.Pred)

0.9998923764685851

In [40]:
SMAPE(compare_val=compareTrain)

SMAPE :  0.06522596017108509
